In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import netCDF4
import xarray
# import rioxarray
import scipy
import matplotlib.pyplot as plt
from glob import glob
import cartopy.crs as ccrs
import scipy.special as sse
from scipy.optimize import curve_fit
from datetime import datetime
import os

In [ ]:
import importlib

import emg_fit
import rotation
import ship_sector_curve

importlib.reload(emg_fit)
importlib.reload(rotation)
importlib.reload(ship_sector_curve)

In [ ]:
# user = 'stijn'
user = 'bart'

if user == 'stijn':
    home = '/Users/stijn/Documents/jaar 2/UC/'
    figuredir = home+'figures/'
    if not os.path.exists(figuredir):
        os.makedirs(figuredir)
    
    NO2_regrid_fp = 'data/regridded_tropomi/'
    NO2_regrid_fns = sorted(glob(NO2_regrid_fp+'*.nc'))
    
    ship_registry_fn = 'C:/Users/stijn/Documents/jaar 2/UC/projectdata/Vessels4AVES.csv'
    AIS_fp = 'C:/Users/stijn/Documents/jaar 2/UC/projectdata/shiptracks/'
elif user == 'bart':
    figuredir = 'figures/'
    if not os.path.exists(figuredir):
        os.makedirs(figuredir)
    
    NO2_regrid_fp = 'data/regridded_tropomi/'
    NO2_regrid_fns = sorted(glob(NO2_regrid_fp+'*.nc'))
    
    ship_registry_fn = 'data/Vessels4AVES.csv'
    AIS_fp = 'data/AIS/AllShipTracks_EasternMed_201906/'

In [ ]:
def clip_to_ship(no2_rotated, no2e_rotated, angle, ship_pixel_first, ship_pixel_last, buffer_length=[3,3]):
    ''' Clip to a box around the ship track '''

    # convert original plume coordinates to the rotated pixel coordinates 
    dy = ship_pixel_last[1] - ship_pixel_first[1]
    dx = ship_pixel_last[0] - ship_pixel_first[0]
    cross_plume = int( dy * np.cos(np.deg2rad(angle)) + dx * np.sin(np.deg2rad(angle)) ) # this should be approximately 1 (the width of the plume)
    along_plume = int( dy * np.sin(np.deg2rad(angle)) - dx * np.cos(np.deg2rad(angle)) ) 
    
    ship_pixel_first_rot = [no2_rotated.shape[1]//2-1, no2_rotated.shape[0]//2-1] # get the center pixel in the rotated image; this is the pixel that is closest to the ship's first position

    # clip to a box around the plume; take buffer_length pixels to get background and allow for wind uncertainty
    no2_rotated_clipped = no2_rotated[ship_pixel_first_rot[1]-cross_plume-buffer_length[1]:ship_pixel_first_rot[1]+cross_plume+buffer_length[1]+1,
                                      ship_pixel_first_rot[0]-along_plume-buffer_length[0]:ship_pixel_first_rot[0]+buffer_length[0]+1] 
    no2e_rotated_clipped = no2e_rotated[ship_pixel_first_rot[1]-cross_plume-buffer_length[1]:ship_pixel_first_rot[1]+cross_plume+buffer_length[1]+1, 
                                        ship_pixel_first_rot[0]-along_plume-buffer_length[0]:ship_pixel_first_rot[0]+buffer_length[0]+1] 

    return no2_rotated_clipped, no2e_rotated_clipped

def clip_to_obs(no2_xds, obs_box):
    ''' Clip the observation to the box; separate uncertainty from data '''
    no2_sliced_xds = no2_xds.tropospheric_NO2_column_number_density.sel(latitude=slice(*obs_box['latitude']), longitude=slice(*obs_box['longitude']))
    no2e_sliced_xds = no2_xds.tropospheric_NO2_column_number_density_uncertainty.sel(latitude=slice(*obs_box['latitude']), longitude=slice(*obs_box['longitude']))
    return no2_sliced_xds, no2e_sliced_xds

def plotwind(ax, northvec, eastvec, gridlon, gridlat, step=10, **kwargs):
    ''' Plot wind vectors on top of the image
        northvec, eastvec: wind vectors
        gridlon, gridlat: lon and lat coordinates of the wind vectors
    '''
    eastwind = np.array(eastvec)[0,:,:]
    northwind = np.array(northvec)[0,:,:]
    lon_arrows = np.repeat(gridlon, len(gridlat), axis=0).reshape(len(gridlon), len(gridlat)).T
    lat_arrows = np.repeat(gridlat, len(gridlon), axis=0).reshape(len(gridlat), len(gridlon))

    # only plot every nth arrow
    eastwind = eastwind[::step,::step]
    northwind = northwind[::step,::step]
    lon_arrows = lon_arrows[::step,::step]
    lat_arrows = lat_arrows[::step,::step]
    ax.quiver(lon_arrows, lat_arrows, eastwind, northwind, color='black', 
                **kwargs) 

### AIS processing

In [ ]:
def GetTime(NO2_filename):
    ''' Get the time of the TROPOMI flyover from the filename '''
    
    TROPOMI_flyover_str = NO2_filename[43:58]
    TROPOMI_flyover_unix = int(datetime.strptime(TROPOMI_flyover_str, '%Y%m%dT%H%M%S').timestamp())
    # print (f"TROPOMI flies over at {TROPOMI_flyover_str} (or {TROPOMI_flyover_unix} seconds after 01-01-1970 00:00 UTC)")

    # convert tropomi flyover str to day in june
    day_in_june = int(TROPOMI_flyover_str[6:8])

    return TROPOMI_flyover_unix, day_in_june

def ProcessAIS(day_in_june, TROPOMI_flyover_unix, box, northwind, eastwind, use_any):
    ''' Process the AIS data
        Returns the filtered vessel MMSI and the AIS_handler object '''
    import get_ais 
    import importlib
    importlib.reload(get_ais) # reload the module to make sure changes are applied

    # create AIS_handler object and find all vessels in AIS data
    AIS_handler = get_ais.AIS(ship_registry_file=ship_registry_fn, ais_folder_path=AIS_fp, 
                              day_of_june2019=day_in_june, TROPOMI_flyover_unix=TROPOMI_flyover_unix)
    vessels = AIS_handler.GetVessels(matchingshipregistry=False) # matchingshipregistry=True to only get vessels that are also in the ship registry

    # filter vessels based on length; could also filter based on other parameters - just add them to the filters dictionary
    AIS_handler.MergeData(vessels=vessels)
    filters = { 'minimum':{'LENGTH REGISTERED':150} } # , 'maximum':{'LENGTH REGISTERED':3500} }
    # filters = {  } # , 'maximum':{'LENGTH REGISTERED':3500} }
    property_filtered_vessels = AIS_handler.FilterVesselProperties(vessels=vessels, filters=filters)

    lat_min, lat_max = box['latitude']
    lon_min, lon_max = box['longitude']

    # perform initial location filtering so ShiftAIS does not have to loop over all vessels
    location_property_filtered_vessels = AIS_handler.FilterVesselLocations(vessels=property_filtered_vessels, lat_min=lat_min, lat_max=lat_max, lon_min=lon_min, lon_max=lon_max,use_any=use_any)
    AIS_handler.ShiftAIS(northwind=northwind, eastwind=eastwind, vessels=location_property_filtered_vessels) # shift the AIS data to the TROPOMI flyover time
    # filter out vessels that do not have all wind shifted data in the box
    location_property_filtered_vessels = AIS_handler.FilterVesselLocations(vessels=location_property_filtered_vessels, lat_min=lat_min, lat_max=lat_max, lon_min=lon_min, lon_max=lon_max,use_any=False)

    return location_property_filtered_vessels, AIS_handler
    
def FilterObservation(NO2_column, box):
    ''' Filter the observation based on whether there is enough data in the box;
        Returns True if the observation should be skipped, False if it should be used '''
    no2_xds_inbox = NO2_column.sel(latitude=slice(*box['latitude']), longitude=slice(*box['longitude']))
    if no2_xds_inbox.data[no2_xds_inbox.data > 0].size < 0.5*no2_xds_inbox.data.size:
        return True # skip the observation
    return False

def red_chisq(model,data,dataerr,k):
    '''Calculate reduced chi square with k degrees of freedom (len(data)-#parameters)'''
    return np.nansum(((model-data)/dataerr)**2)/k



### Perform EMG fit

In [ ]:
# rotate the image so that the wind vectors are aligned with the x and y axis

pixelsize_lon = 4200
pixelsize_lat = 5000
obs_box = {'latitude': (31.5, 34.2), 'longitude': (19.5, 29.5)} # box to filter observations
buffer_length = [3,8] # along plume and across plume buffer length in pixels, respectively

In [ ]:
emission_rates = []
lifetimes = []
MMSIs = []
velocities = []

allship_count = 0
allship_accepted = 0 # to get the ratio of accepted fits to all attempted fits

for i, NO2_file in enumerate(NO2_regrid_fns):

    # if '20190621' not in NO2_file:
    #     continue
    
    NO2_xds = xarray.open_dataset(NO2_file, engine='netcdf4')

    # get AIS data for vessels in the study region, filter out NO2 observations with largely NaN in a given latitude, longitude box
    TROPOMI_flyover_unix, day_in_june = GetTime(NO2_file)
    filtered_vessels, AIS_handler = ProcessAIS(day_in_june, TROPOMI_flyover_unix, box=obs_box,
                                            northwind=NO2_xds.surface_meridional_wind_velocity, eastwind=NO2_xds.surface_zonal_wind_velocity,
                                            use_any=True)
    if FilterObservation(NO2_xds.tropospheric_NO2_column_number_density, obs_box):
        continue

    # get vectors of the ship tracks and NO2 data in the box
    trackvector_dict = AIS_handler.GetTrackVector(filtered_vessels, *obs_box['latitude'], *obs_box['longitude'])
    no2_sliced_xds, no2e_sliced_xds = clip_to_obs(NO2_xds, obs_box)

    for mmsi, trackvector in trackvector_dict.items(): # loop over all vessels in the box

        angle = np.rad2deg(np.pi-np.arctan2(trackvector[1], trackvector[0])) 

        # rotate the image so that the plume is aligned with the x axis
        ship_pixel_first, ship_pixel_last = rotation.get_pivot_pixel(AIS_handler, mmsi, no2_sliced_xds)
        no2_rotated, no2e_rotated = rotation.perform_rotation(no2_sliced_xds.data[0,:,:], no2e_sliced_xds.data[0,:,:], angle, pivot=ship_pixel_first)
        no2_rotated_clipped, no2e_rotated_clipped = clip_to_ship(no2_rotated, no2e_rotated, angle, ship_pixel_first, ship_pixel_last, buffer_length=buffer_length)

        # get data in ship sector
        uncertainty_line = ship_sector_curve.get_uncertainty_line(np.arange(no2_rotated_clipped.shape[1]-2*buffer_length[0]), wind_uncertainty=20)
        new_ship_pixel_first = [no2_rotated_clipped.shape[0]//2, no2_rotated_clipped.shape[1]-buffer_length[0]] # ship pixel in rotated, clipped image
        new_ship_pixel_last = [no2_rotated_clipped.shape[0]//2, buffer_length[0]]
        no2_rotated_sectored = ship_sector_curve.plume_triangle_mask(no2_rotated_clipped, uncertainty_line, new_ship_pixel_first, new_ship_pixel_last)
        no2e_rotated_sectored = ship_sector_curve.plume_triangle_mask(no2e_rotated_clipped, uncertainty_line, new_ship_pixel_first, new_ship_pixel_last)

        maxvalues, maxvalueserr, indices = ship_sector_curve.get_plume_curve(no2_rotated_clipped, no2e_rotated_clipped, no2_rotated_sectored, no2e_rotated_sectored, uncertainty_line, buffer_length)
        if maxvalues is None:
            continue

        # Normalise errors to a more feasible level by fitting the function, calculating rchisq and dividing the errors by it
        # p0 = [mu_x, sigma_x, E, x_0, B]
        p0 = [5, 10, 1e-5, 10, np.nanmin(maxvalues)] # initial guess for the fit, in units of pixels. Most important here is to get the order of magnitude
        popt, pcov = emg_fit.EMG_fit(indices, maxvalues, yerr=maxvalueserr, p0=p0, absolute_sigma=True)

        if popt is None:
            continue
        rchisq = red_chisq(emg_fit.EMG(indices,*popt),maxvalues,maxvalueserr,len(indices)-len(popt))
        maxvalueserr *= np.sqrt(rchisq)
        # refit the EMG function to the data
        popt, pcov = emg_fit.EMG_fit(indices, maxvalues, yerr=maxvalueserr, p0=popt, absolute_sigma=True)
        if popt is None:
            continue

        # calculate pixel size after rotation
        pixelsize_x, pixelsize_y = rotation.get_pixelsize(pixelsize_lat, pixelsize_lon, angle, no2_sliced_xds)
        
        # calculate plume parameters
        w = AIS_handler.GetAvgShiftedShipSpeed(mmsi) / pixelsize_x # combined speed of ship and wind in along plume pixels/s
        (tau_EMG,tau_err_EMG), (E_EMG, E_err_EMG) = emg_fit.PlumeParams(popt, pcov, w)
        # print(f'tau_EMG = {tau_EMG/3600:.2f} +- {tau_err_EMG/3600:.2f} hours') # seconds

        NO2_mass = E_EMG * pixelsize_x * pixelsize_y * 46.0055 # NO2 mass in g/s
        NO2_mass_err = E_err_EMG * pixelsize_x * pixelsize_y * 46.0055 # NO2 mass in g/s
        # print(f'NO2 mass = {NO2_mass/1000:.4g} +- {NO2_mass_err/1000:.2g} kg/s')
         
        # set conditions for accepting the fit
        conditions = [tau_err_EMG / tau_EMG < 1, # relative error on tau_EMG < 1
                      E_err_EMG / E_EMG < 1, # relative error on E_EMG < 1
                      all([np.sqrt(pcov[i,i])/popt[i] < 1 for i in range(len(popt))]), # relative error on all parameters < 1
                      tau_err_EMG/3600 > 0.1, # lifetime error > 0.1 hours
                      popt[0] < 8, # peak position < 8 pixels
                      ]

        allship_count += 1
        if all(conditions): # the fit is accepted
            allship_accepted += 1

            emission_rates.append(NO2_mass)
            lifetimes.append(tau_EMG)
            MMSIs.append(mmsi)
            velocities.append(AIS_handler.GetAvgShipSpeed_noshift(mmsi))
            # plotting

            fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10,12), sharex=True, width_ratios=[1])
            # select only rows with data
            no2_rotated_sectored = no2_rotated_sectored[~np.all(np.isnan(no2_rotated_sectored), axis=1)]
            img = ax[0].pcolormesh(np.arange(no2_rotated_sectored.shape[1])*pixelsize_x/1000, np.arange(no2_rotated_sectored.shape[0])*pixelsize_y/1000, no2_rotated_sectored/1e-5, cmap='Blues', vmin=0)
            ax[0].set_aspect('equal', adjustable='box')
            ax[0].set_ylabel('distance across shifted track [km]', fontsize=16)

            ax[1].errorbar(indices*pixelsize_x/1000, maxvalues/1e-5, yerr=maxvalueserr/1e-5, fmt='o', label='plume data',capsize=2)
            ax[1].plot(np.linspace(0, indices[-1], 1000)*pixelsize_x / 1000, emg_fit.EMG(np.linspace(0, indices[-1], 1000), *popt)/1e-5, 'r-', label='EMG fit')
            ax[1].set_xlabel('distance along shifted track [km]', fontsize=16)
            ax[1].set_ylabel('NO2 column density [$10^{-5}$ mol/m$^2$]', fontsize=16)
            ax[1].legend(fontsize=14)

            ax[0].tick_params(labelsize=12)
            ax[1].tick_params(labelsize=12)

            ax[1].text(0.05, 1.01, f'$\\tau$ = {tau_EMG/3600:.1f} +- {tau_err_EMG/3600:.1f} hours\nE = {NO2_mass:.3g} +- {NO2_mass_err:.1g} g NO$_2$/s', transform=ax[1].transAxes, fontsize=16)

            # decrease vertical spacing between subplots
            plt.subplots_adjust(hspace=0.05)
    
            # add colorbar beside axes, while retaining shared x axis
            fig.subplots_adjust(right=0.8)
            cbar_ax = fig.add_axes([0.83, 0.55, 0.02, 0.35])
            bar = fig.colorbar(img, cax=cbar_ax, label='Tropospheric NO2 column number density [$10^{-5}$ mol/m$^2$]')
            bar.ax.yaxis.label.set_size(15)
            bar.ax.tick_params(labelsize=12)
            # plt.tight_layout()
            plt.savefig(figuredir + f'/fits_withparams/june{day_in_june}_iter{allship_accepted}_fit.pdf')
            plt.close()
    NO2_xds.close()
    # break

emission_rates = np.array(emission_rates)
lifetimes = np.array(lifetimes) / 3600 # convert to hours
MMSIs = np.array(MMSIs,dtype=int)
velocities = np.array(velocities)

#### Produce nice histograms figures now!

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,5))

ax[0].hist(np.array(emission_rates), bins=np.linspace(0,max(emission_rates),60),color='k',histtype='step',linewidth=2)
ax[0].set_xlim(-2,280)
# ax[0].hist(np.array(emission_rates)*1.32, bins=np.logspace(np.log10(min(emission_rates*1.32)),np.log10(max(emission_rates*1.32)),20))
# ax[0].set_xscale('log')
ax[0].set_xlabel('NO$_2$ Emission rate [g/s]',fontsize=18)
# ax[0].text(0.67, 0.88, f'average = {np.nanmean(emission_rates*1.32):.2g} g/s\nmedian = {np.nanmedian(emission_rates*1.32):.2g} g/s', transform=ax[0].transAxes, fontsize=14)
avg,med = np.nanmean(emission_rates),np.nanmedian(emission_rates)
# ax[0].axvline(avg,c='r',ls=':',label=f'Average = {avg:.3g} g/s',linewidth=2)
ax[0].axvline(med,c='b',ls='--',label=f'Median = {med:.3g} g/s',linewidth=2)

ax[1].hist(np.array(lifetimes), bins=20,color='k',histtype='step',linewidth=2)
ax[1].set_xlabel('Lifetime [hrs]',fontsize=18)
# ax[1].text(0.58, 0.88, f'average = {np.nanmean(lifetimes):.2f} hours\nmedian = {np.nanmedian(lifetimes):.2f} hours', transform=ax[1].transAxes, fontsize=14)
avg,med = np.nanmean(lifetimes),np.nanmedian(lifetimes)
# ax[1].axvline(avg,c='r',ls=':',label=f'Average = {avg:.3g} hrs',linewidth=2)
ax[1].axvline(med,c='b',ls='--',label=f'Median = {med:.3g} hrs',linewidth=2)
ax[1].axvline(x=3, color='red', linestyle='-', label='\'Upper limit\'',linewidth=2)
ax[1].set_yticks(range(0,21,5))
for i in range(2):
    ax[i].legend(loc=1,fontsize=14)
    ax[i].set_ylabel("Counts",fontsize=18)
    ax[i].tick_params(axis='both', which='major', labelsize=14)

plt.tight_layout()
plt.savefig(figuredir + f'/histograms.pdf')
plt.show()

In [ ]:
print(f'{allship_accepted/allship_count*100:.2f}% of the fits were accepted')
print(allship_accepted)

### Calculate emission proxy for vessels in ship registry

In [ ]:
ship_registry = pd.read_csv(ship_registry_fn)
ship_registry.head()

In [ ]:
lengths = np.zeros(len(MMSIs))
# MMSIs
for i,mmsi in enumerate(MMSIs):
    msk = ship_registry['MMSI'] == mmsi
    try:
        lengths[i] = np.array(ship_registry['LENGTH REGISTERED'])[msk][0]
    except:
        lengths[i] = np.nan

emission_proxy = (lengths**2) * (velocities**3)

In [ ]:
print ("Missing data",np.sum(np.isnan(lengths))/len(lengths), "N=",len(lengths)-np.sum(np.isnan(lengths)))

# plt.scatter(lengths,emission_rates)
# plt.xlabel("Length [m]")
# plt.ylabel("NO$_2$ emission rate [g/s]")
# plt.show()
nonan = (~np.isnan(emission_proxy)) * (velocities>1.83)
print ("Ships moving too slow",np.sum((velocities<1.83)))
print ("Total N after selection", np.sum(nonan))
plt.figure(figsize=(6,3))
plt.scatter(emission_proxy[nonan],emission_rates[nonan],c='k',label='datapoints')
plt.ylabel("NO$_2$ emission rate [g/s]",fontsize=14)
plt.xlabel("Ship emission proxy [m$^5$/s$^3$]",fontsize=16)
plt.xticks(fontsize=12); plt.yticks(fontsize=12)
def linear(x,a,b):
    return a*x+b

popt,pcov = curve_fit(linear,emission_proxy[nonan],emission_rates[nonan],p0=[4e-7,5])
tmp = np.linspace(0,max(emission_proxy[nonan]),100)
plt.plot(tmp,linear(tmp,*popt),c='r',label='Linear fit')
plt.legend(loc=2,fontsize=14)

from scipy.stats import pearsonr
correlation = pearsonr(emission_proxy[nonan],emission_rates[nonan])[0]
plt.gca().text(0.43,0.88,'$\\rho$'+f' = {correlation:.3f}',transform=plt.gca().transAxes, fontsize=16)
plt.tight_layout()
plt.savefig("figures/Emission_proxy.pdf")
plt.show()